In [ ]:
!pip install torch torchvision
!pip install transformers datasets
!pip install transformers[torch]
!pip install matplotlib
!pip install scikit-learn
!pip install --upgrade transformers

In [ ]:
# ============================================================================
# CELL 2: IMPORTS - VERSÃO PARA DATASET CUSTOMIZADO
# ============================================================================
# ALTERAÇÃO: Adicionamos pandas, os, e Dataset do PyTorch
# RAZÃO: Necessários para carregar dados de CSV e criar dataset customizado
# COMPARAÇÃO: Antes usávamos datasets.load_dataset (HuggingFace). Agora:
#   - pandas: para ler/manipular CSV
#   - os: para navegar sistema de arquivos
#   - Dataset: base para criar CustomImageDataset

# PyTorch
import torch
import torchvision
from torchvision.transforms import Normalize, Resize, ToTensor, Compose
# For displaying images
from PIL import Image
import matplotlib.pyplot as plt
from torchvision.transforms import ToPILImage
# Loading dataset - ALTERADO para CustomImageDataset
import pandas as pd
import os
from torch.utils.data import Dataset, DataLoader, random_split
# Transformers
from transformers import ViTImageProcessor, ViTForImageClassification
from transformers import TrainingArguments, Trainer
# Matrix operations
import numpy as np
# Evaluation
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# ============================================================================
# CÓDIGO ANTIGO (COMENTADO) - Para referência e comparação
# ============================================================================
# # Loading dataset - ORIGINAL
# from datasets import load_dataset

In [ ]:
# ============================================================================
# CELL 3: CARREGAR E PREPARAR DATASET - VERSÃO CUSTOMIZADA
# ============================================================================
# ALTERAÇÃO PRINCIPAL: Substituir load_dataset por CustomImageDataset
# RAZÃO: Em cenários reais, os dados estão em CSV + pastas de imagens
# ESTRUTURA DE DADOS:
#   /workspace/data/train.csv → file_id,class (14035 samples com labels)
#   /workspace/data/train/*.jpg → imagens
#   /workspace/data/test.csv → file_id (4945 samples SEM labels - cenário real)
#   /workspace/data/test/*.jpg → imagens para predição

# ============================================================================
# CLASSE CUSTOMIZADA: CustomImageDataset
# ============================================================================
# RAZÃO: Permite flexibilidade total no carregamento de dados customizados
# VANTAGENS vs Hugging Face datasets:
#   1. Simples de entender e modificar
#   2. Suporta dados sem labels (test set real)
#   3. Fácil adicionar novas imagens (só alterar CSV)

class CustomImageDataset(Dataset):
    """
    Dataset customizado para carregar imagens de pastas com metadata em CSV
    
    Args:
        csv_file (str): Caminho para arquivo CSV com colunas [file_id, class]
        img_dir (str): Diretório contendo as imagens
        transform (callable, optional): Transformações a aplicar nas imagens
        has_labels (bool): Se True, CSV tem coluna 'class'. Se False, apenas 'file_id'
    """
    def __init__(self, csv_file, img_dir, transform=None, has_labels=True):
        self.df = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform
        self.has_labels = has_labels
        
        # Descobrir classes automaticamente a partir dos dados
        if self.has_labels:
            self.classes = sorted(self.df['class'].unique())
            self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}
        else:
            self.classes = []
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        file_id = self.df.iloc[idx]['file_id']
        img_path = os.path.join(self.img_dir, f'{file_id}.jpg')
        
        # Carregar imagem
        image = Image.open(img_path).convert('RGB')
        
        # Aplicar transformações se fornecidas
        if self.transform:
            pixels = self.transform(image)
        else:
            pixels = image
        
        # Retornar dados
        if self.has_labels:
            label = self.df.iloc[idx]['class']
            return {'pixels': pixels, 'label': label, 'img': image, 'file_id': file_id}
        else:
            # Para test set: sem label, apenas ID e pixels
            return {'pixels': pixels, 'file_id': file_id, 'img': image}

# ============================================================================
# CARREGAR DADOS CUSTOMIZADOS
# ============================================================================
# CENÁRIO REAL: 
#   - train.csv tem labels → usado para treinar
#   - test.csv NÃO tem labels → usado para submissão Kaggle

# Caminho dos dados
TRAIN_CSV = '/workspace/data/train.csv'
TEST_CSV = '/workspace/data/test.csv'
TRAIN_IMG_DIR = '/workspace/data/train'
TEST_IMG_DIR = '/workspace/data/test'

# Carregar datasets com labels (para treino)
full_dataset = CustomImageDataset(
    csv_file=TRAIN_CSV, 
    img_dir=TRAIN_IMG_DIR, 
    transform=None,  # Aplicaremos transformações depois
    has_labels=True
)

# Dividir em treino e validação (estratificado para manter proporções)
# IMPORTANTE: Isso garante que train/val tenham mesma distribuição de classes
train_size = int(0.9 * len(full_dataset))
val_size = len(full_dataset) - train_size

trainds, valds = random_split(full_dataset, [train_size, val_size])

# Carregar test set SEM labels (cenário real de competição Kaggle)
testds = CustomImageDataset(
    csv_file=TEST_CSV,
    img_dir=TEST_IMG_DIR,
    transform=None,
    has_labels=False  # IMPORTANTE: test não tem labels em competições
)

print(f"Train samples: {len(trainds)}")
print(f"Val samples: {len(valds)}")
print(f"Test samples: {len(testds)}")
print(f"Classes identificadas: {full_dataset.classes}")
print(f"Número de classes: {len(full_dataset.classes)}")

# ============================================================================
# CÓDIGO ANTIGO (COMENTADO) - Para referência
# ============================================================================
# # ORIGINAL - Usando Hugging Face datasets:
# trainds, testds = load_dataset("cifar10", split=["train[:5000]","test[:1000]"])
# splits = trainds.train_test_split(test_size=0.1)
# trainds = splits['train']
# valds = splits['test']
# trainds, valds, testds

In [ ]:
# ============================================================================
# CELL 4: CRIAR MAPEAMENTOS DE CLASSES (itos/stoi)
# ============================================================================
# ALTERAÇÃO: Agora classes vêm automaticamente do dataset customizado
# RAZÃO: Em dados reais, o número de classes é dinâmico
# COMPARAÇÃO: Antes usávamos trainds.features['label'].names (Hugging Face)
#            Agora usamos full_dataset.classes (descoberto automaticamente do CSV)

# itos = int-to-string: mapeia índice de classe para nome legível
#   Exemplo: 0 → 'class_0', 1 → 'class_1'
# stoi = string-to-int: mapeamento inverso
#   Exemplo: 'class_0' → 0, 'class_1' → 1

# ESTRUTURA DO DADO: train.csv tem números de classe (0, 1, 2, ...)
# Criamos nomes legíveis associados

class_names = [f'class_{i}' for i in full_dataset.classes]
itos = dict(enumerate(class_names))
stoi = {v: k for k, v in itos.items()}

print("Mapeamento INT-TO-STRING (itos):")
print(itos)
print("\nMapeamento STRING-TO-INT (stoi):")
print(stoi)
print(f"\nTotal de classes: {len(itos)}")

# ============================================================================
# CÓDIGO ANTIGO (COMENTADO) - Para referência
# ============================================================================
# # ORIGINAL - Usando Hugging Face naming:
# itos = dict((k,v) for k,v in enumerate(trainds.features['label'].names))
# stoi = dict((v,k) for k,v in enumerate(trainds.features['label'].names))
# itos, stoi

In [ ]:
# ============================================================================
# CELL 5: VISUALIZAR AMOSTRA DO DATASET - ANTES DE TRANSFORMAÇÕES
# ============================================================================
# ALTERAÇÃO: Agora trabalhamos com dataset customizado que retorna dicts
# RAZÃO: Estrutura diferente de Hugging Face datasets
# IMPORTANTE: Visualizar ANTES das transformações para ver imagem original

# Pegar primeiro sample do dataset de treino (sem transformações ainda)
train_dataset_raw = full_dataset  # Dataset bruto, sem transformações

index = 0
sample = train_dataset_raw[index]

print(f"Estrutura do sample retornado:")
print(f"  - pixels: tipo {type(sample['pixels'])}")
print(f"  - label: {sample['label']}")
print(f"  - file_id: {sample['file_id']}")
print(f"  - classe interpretada: {class_names[sample['label']]}")

# Exibir imagem
img = sample['img']
plt.figure(figsize=(4, 4))
plt.imshow(img)
plt.title(f"Classe: {class_names[sample['label']]} (ID do arquivo: {sample['file_id']})")
plt.axis('off')
plt.show()

# ============================================================================
# CÓDIGO ANTIGO (COMENTADO) - Para referência
# ============================================================================
# # ORIGINAL:
# index = 0
# img, lab = trainds[index]['img'], itos[trainds[index]['label']]
# print(lab)
# img

In [ ]:
# ============================================================================
# CELL 6: CARREGAR PROCESSADOR E EXTRAIR PARÂMETROS DE NORMALIZAÇÃO
# ============================================================================
# SEM ALTERAÇÕES: Este código continua o mesmo
# RAZÃO: O processador do ViT é universal e funciona igual
# IMPORTANTE: ImageNet normalization é crítica para transfer learning

model_name = "google/vit-base-patch16-224"
processor = ViTImageProcessor.from_pretrained(model_name) 

mu, sigma = processor.image_mean, processor.image_std  # Parâmetros ImageNet
size = processor.size

print(f"Modelo: {model_name}")
print(f"Tamanho de entrada: {size}")
print(f"Média de normalização (ImageNet): {mu}")
print(f"Desvio padrão (ImageNet): {sigma}")
print("\nPORQUÊ ImageNet stats? Transfer learning depende que input distribution")
print("corresponda exatamente ao que o modelo foi treinado. Usar outros valores")
print("causaria catastrophic forgetting dos pesos pré-treinados.")

In [ ]:
# ============================================================================
# CELL 7: DEFINIR PIPELINE DE TRANSFORMAÇÕES
# ============================================================================
# ALTERAÇÃO IMPORTANTE: Usar Compose corretamente para garantir tamanho fixo
# PROBLEMA ORIGINAL: Resize sem CenterCrop pode manter aspect ratio
# SOLUÇÃO: Forçar tamanho exato 224x224 com CenterCrop

from torchvision.transforms import CenterCrop

norm = Normalize(mean=mu, std=sigma)  # Normalizar para [-1,1]

# Pipeline de transformação CORRIGIDO:
#   1. Resize(256): Redimensiona mantendo aspect ratio (lado menor = 256)
#   2. CenterCrop(224): Recorta centro para 224x224 (GARANTE TAMANHO FIXO!)
#   3. ToTensor(): PIL Image → Tensor PyTorch
#   4. Normalize(): Aplicar mean/std ImageNet
_transf = Compose([
    Resize(256),              # Redimensiona lado menor para 256 (mantém aspect ratio)
    CenterCrop(224),          # ✅ NOVO: Garante exatamente 224x224
    ToTensor(),
    norm
]) 

print("Pipeline de transformações (CORRIGIDO):")
print("  1. Resize(256) - redimensiona mantendo aspect ratio")
print("  2. CenterCrop(224) - ✅ NOVO: garante tamanho uniforme 224x224")
print("  3. ToTensor() - converter PIL Image para torch.Tensor")
print("  4. Normalize(mu, sigma) - normalizar com ImageNet stats")
print("  5. Output: tensor 3x224x224 com valores em [-1, 1]")
print("\n💡 Por quê CenterCrop? Imagens com aspect ratios diferentes precisam")
print("   ser recortadas para tamanho uniforme antes de fazer stack em batches")

In [ ]:
# ============================================================================
# CELL 8: APLICAR TRANSFORMAÇÕES AO DATASET
# ============================================================================
# ALTERAÇÃO: CustomImageDataset pode receber transforms no __init__
# RAZÃO: Mais eficiente que usar set_transform (que é método de HF Dataset)
# ESTRATÉGIA: 
#   - Criar "wrapper" que aplica transformações no __getitem__
#   - Train/Val: aplicar transformações
#   - Test: aplicar transformações (para que shape fique correto)

class TransformDataset(Dataset):
    """Wrapper que aplica transformações a um dataset existente"""
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        sample = self.dataset[idx]
        if self.transform:
            sample['pixels'] = self.transform(sample['img'])
        else:
            sample['pixels'] = sample['img']
        return sample

# Aplicar transformações aos datasets
trainds = TransformDataset(trainds, transform=_transf)
valds = TransformDataset(valds, transform=_transf)
testds = TransformDataset(testds, transform=_transf)

print("Transformações aplicadas a:")
print(f"  - Train set ({len(trainds)} amostras)")
print(f"  - Val set ({len(valds)} amostras)")
print(f"  - Test set ({len(testds)} amostras - SEM labels)")

# ============================================================================
# CÓDIGO ANTIGO (COMENTADO) - Para referência
# ============================================================================
# # ORIGINAL - Usando método set_transform (Hugging Face):
# # apply transforms to PIL Image and store it to 'pixels' key
# def transf(arg):
#     arg['pixels'] = [_transf(image.convert('RGB')) for image in arg['img']]
#     return arg
# 
# trainds.set_transform(transf)
# valds.set_transform(transf)
# testds.set_transform(transf)

In [ ]:
# ============================================================================
# CELL 9: VISUALIZAR IMAGEM APÓS TRANSFORMAÇÕES
# ============================================================================
# SEM ALTERAÇÕES SIGNIFICATIVAS: Visualização funciona igual
# NOTA: Denormalizamos para poder visualizar (valores voltam a [0,1])

idx = 0
sample = trainds[idx]
ex = sample['pixels']  # Tensor normalizado

# Denormalizar para visualizar (reverter Normalize operation)
# Fórmula: x_original = (x_normalizado * sigma) + mu
ex_denorm = (ex * torch.tensor(sigma).view(3, 1, 1)) + torch.tensor(mu).view(3, 1, 1)
ex_denorm = torch.clamp(ex_denorm, 0, 1)  # Manter em [0, 1]

# Converter para PIL Image e exibir
exi = ToPILImage()(ex_denorm)
plt.figure(figsize=(4, 4))
plt.imshow(exi)
plt.title(f"Imagem após transformações - Classe: {class_names[sample['label']]}")
plt.axis('off')
plt.show()

print(f"Shape do tensor: {ex.shape}")
print(f"Valor mínimo: {ex.min():.4f}, Máximo: {ex.max():.4f}")
print("✓ Imagem normalizada corretamente (valores em [-1, 1])")

In [ ]:
# ============================================================================
# CELL 10: CARREGAR MODELO PRÉ-TREINADO (SEM ADAPTAÇÃO)
# ============================================================================
# SEM ALTERAÇÕES: Carregar modelo em sua forma original
# RAZÃO: Próximo passo será adaptar para o número correto de classes

model_name = "google/vit-base-patch16-224"
model = ViTForImageClassification.from_pretrained(model_name)

print(f"Modelo original carregado: {model_name}")
print(f"Camada de classificação original (1000 classes ImageNet):")
print(model.classifier)

In [ ]:
# ============================================================================
# CELL 11: ADAPTAR MODELO PARA NÚMERO CORRETO DE CLASSES
# ============================================================================
# ALTERAÇÃO: usar len(full_dataset.classes) em vez de hardcoded 10
# RAZÃO: Torna código genérico para qualquer número de classes
# IMPORTANTE: ignore_mismatched_sizes=True permite redimensionar classifier layer

num_labels = len(full_dataset.classes)

model = ViTForImageClassification.from_pretrained(
    model_name, 
    num_labels=num_labels,  # Dinâmico baseado nos dados
    ignore_mismatched_sizes=True,  # Permite adaptar 1000→num_labels
    id2label=itos,  # Mapeamento índice → nome
    label2id=stoi   # Mapeamento nome → índice
)

print(f"Modelo adaptado para {num_labels} classes")
print(f"Camada de classificação adaptada:")
print(model.classifier)
print(f"\nMapeamentos configurados:")
print(f"  id2label: {itos}")
print(f"  label2id: {stoi}")

# ============================================================================
# CÓDIGO ANTIGO (COMENTADO) - Para referência
# ============================================================================
# # ORIGINAL - Hardcoded para CIFAR-10 (10 classes):
# model = ViTForImageClassification.from_pretrained(
#     model_name, 
#     num_labels=10,  # Hardcoded
#     ignore_mismatched_sizes=True, 
#     id2label=itos, 
#     label2id=stoi
# )
# print(model.classifier)

In [ ]:
# ============================================================================
# CELL 12: CONFIGURAR ARGUMENTOS DE TREINAMENTO
# ============================================================================
# ALTERAÇÃO: Otimizar batch_size e epochs para treinamento mais rápido
# RAZÃO: Batch size original (10) era muito pequeno
# ESTRATÉGIA: Se GPU disponível, usar batch_size maior; senão, manter pequeno

# Detectar se tem GPU para otimizar
has_gpu = torch.cuda.is_available()

# Otimizar batch size baseado em hardware
if has_gpu:
    train_batch_size = 32  # GPU pode lidar com batches maiores
    eval_batch_size = 64   # Avaliação pode ser ainda maior
    num_epochs = 3
    print("🚀 GPU detectada! Usando batch sizes otimizados")
else:
    train_batch_size = 10  # CPU precisa de batches menores
    eval_batch_size = 10
    num_epochs = 3
    print("⚠️  GPU NÃO detectada. Usando batch sizes para CPU (treinamento será lento)")

args = TrainingArguments(
    f"test-custom-dataset",
    save_strategy="epoch",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=train_batch_size,  # ✅ OTIMIZADO
    per_device_eval_batch_size=eval_batch_size,    # ✅ OTIMIZADO
    num_train_epochs=num_epochs,
    weight_decay=0.04,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    logging_dir='logs',
    logging_steps=50,  # Log a cada 50 steps (para ver progresso)
    remove_unused_columns=False,
)

print("\n✅ Configuração de treinamento:")
print(f"  - Output dir: {args.output_dir}")
print(f"  - Learning rate: {args.learning_rate}")
print(f"  - Batch size TREINO: {args.per_device_train_batch_size}")
print(f"  - Batch size VALIDAÇÃO: {args.per_device_eval_batch_size}")
print(f"  - Epochs: {args.num_train_epochs}")
print(f"  - Métrica para melhor modelo: {args.metric_for_best_model}")
print(f"  - GPU: {'✅ Sim' if has_gpu else '❌ Não'}")

# ============================================================================
# CÓDIGO ANTIGO (COMENTADO) - Para referência
# ============================================================================
# # ORIGINAL - Hardcoded, batch size muito pequeno:
# args = TrainingArguments(
#     f"test-cifar-10",
#     save_strategy="epoch",
#     eval_strategy ="epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=10,  # ❌ Muito pequeno
#     per_device_eval_batch_size=4,    # ❌ Muito pequeno
#     num_train_epochs=3,
#     weight_decay=0.04,
#     load_best_model_at_end=True,
#     metric_for_best_model="accuracy",
#     logging_dir='logs',
#     remove_unused_columns=False,
# )

In [ ]:
# ============================================================================
# CELL 13: DEFINIR FUNÇÕES DE COLATE E MÉTRICAS
# ============================================================================
# ALTERAÇÃO: collate_fn agora lida com dataset customizado
# RAZÃO: CustomImageDataset retorna dicts com estrutura ligeiramente diferente
# IMPORTANTE: Manter file_id nos batches para rastrear predições no test set

def collate_fn(examples):
    """
    Collate function customizado que transforma lista de examples em batch
    
    RAZÃO: Trainer espera que pixel_values e labels sejam stacked em dimensão batch
    
    INPUT: lista de dicts com keys ['pixels', 'label', 'file_id', 'img']
    OUTPUT: dict com keys ['pixel_values', 'labels', 'file_id']
    """
    # Stack de tensores de pixels (transformando lista em batch)
    pixels = torch.stack([example["pixels"] for example in examples])
    
    # Converter labels para tensor
    labels = torch.tensor([example["label"] for example in examples])
    
    # Manter file_ids para rastreamento (importante para test set)
    file_ids = [example["file_id"] for example in examples]
    
    # Retornar no formato esperado pelo Trainer
    return {
        "pixel_values": pixels,  # IMPORTANTE: nome correto para ViT (não 'pixels')
        "labels": labels,
        "file_id": file_ids  # Adicional: útil para submissões Kaggle
    }

def compute_metrics(eval_pred):
    """
    Computar métricas de avaliação
    
    RAZÃO: Trainer executa isso após cada epoch para monitorar progresso
    
    INPUT: EvalPrediction com predictions (logits brutos) e labels verdadeiros
    OUTPUT: dict com métricas (accuracy, etc)
    """
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)  # Converter logits → classes
    return dict(accuracy=accuracy_score(predictions, labels))

print("✓ Funções collate_fn e compute_metrics definidas")
print("\ncollate_fn:")
print("  - Transforma lista de samples em batch com shapes corretos")
print("  - Renomeia 'pixels' → 'pixel_values' (obrigatório para ViT)")
print("  - Manter file_ids para rastreamento")
print("\ncompute_metrics:")
print("  - Calcula accuracy durante validação")
print("  - Permite monitorar progresso do treinamento")

# ============================================================================
# CÓDIGO ANTIGO (COMENTADO) - Para referência
# ============================================================================
# # ORIGINAL - Sem file_id:
# def collate_fn(examples):
#     pixels = torch.stack([example["pixels"] for example in examples])
#     labels = torch.tensor([example["label"] for example in examples])
#     return {"pixel_values": pixels, "labels": labels}
# 
# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     predictions = np.argmax(predictions, axis=1)
#     return dict(accuracy=accuracy_score(predictions, labels))

In [ ]:
# ============================================================================
# CELL 14: CRIAR TRAINER
# ============================================================================
# SEM ALTERAÇÕES SIGNIFICATIVAS: Trainer funciona igual com datasets customizados
# IMPORTANTE: Trainer automáticamente usa collate_fn que definimos

trainer = Trainer(
    model,
    args, 
    train_dataset=trainds,
    eval_dataset=valds,
    data_collator=collate_fn,  # Nossa função customizada
    compute_metrics=compute_metrics,  # Nossa função de métricas
    tokenizer=processor,  # Processador do ViT
)

print("✓ Trainer criado com sucesso")
print(f"  - Modelo: {model.config.architectures[0]}")
print(f"  - Classes: {num_labels}")
print(f"  - Train batches: {len(trainds) // args.per_device_train_batch_size}")
print(f"  - Val batches: {len(valds) // args.per_device_eval_batch_size}")
print(f"  - Epochs: {args.num_train_epochs}")

# ============================================================================
# CÓDIGO ANTIGO (COMENTADO) - Para referência
# ============================================================================
# # ORIGINAL:
# trainer = Trainer(
#     model,
#     args, 
#     train_dataset=trainds,
#     eval_dataset=valds,
#     data_collator=collate_fn,
#     compute_metrics=compute_metrics,
#     tokenizer=processor,
# )

In [ ]:
# ============================================================================
# CELL 14B: DIAGNÓSTICO E VALIDAÇÃO DE DADOS (NOVO)
# ============================================================================
# RAZÃO: Detectar problemas nos dados ANTES de treinar
# IMPORTANTE: Verificar:
#   1. Imagens faltando no disco
#   2. Tamanhos inconsistentes (causa erro de stack)
#   3. GPU disponível (determina velocidade)
#   4. Estimar tempo de treinamento

import torch

print("="*70)
print("🔍 DIAGNÓSTICO DE DADOS E HARDWARE")
print("="*70)

# 1. Verificar GPU
print("\n1️⃣  VERIFICAR GPU:")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"  Dispositivo disponível: {device}")
if torch.cuda.is_available():
    print(f"  GPU: {torch.cuda.get_device_name(0)}")
    print(f"  VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("  ⚠️  GPU NÃO DISPONÍVEL! Treinamento será MUITO lento (CPU)")

# 2. Verificar integridade das imagens e tamanhos
print("\n2️⃣  VERIFICAR INTEGRIDADE DAS IMAGENS:")

# Verificar train
missing_train = 0
size_issues = 0
train_sizes = {}

print("  Escaneando train dataset...")
for i in range(min(100, len(full_dataset))):  # Verificar primeiras 100
    try:
        sample = full_dataset[i]
        file_id = sample['file_id']
        img = sample['img']
        size = img.size  # (width, height)
        
        if size not in train_sizes:
            train_sizes[size] = 0
        train_sizes[size] += 1
    except Exception as e:
        missing_train += 1
        if missing_train <= 5:  # Mostrar primeiros 5 erros
            print(f"    ❌ Erro no índice {i}: {e}")

print(f"  ✓ Train: {len(full_dataset)} amostras verificadas")
if missing_train > 0:
    print(f"  ⚠️  {missing_train} imagens com problema")
print(f"  Distribuição de tamanhos de imagem (primeiras 100):")
for size, count in sorted(train_sizes.items(), key=lambda x: -x[1])[:5]:
    print(f"    - {size[0]}x{size[1]}: {count} imagens")

# 3. Testar batch com diferentes tamanhos
print("\n3️⃣  TESTAR CRIAÇÃO DE BATCH:")
from torch.utils.data import DataLoader

try:
    # Criar mini batch para testar
    test_loader = DataLoader(
        TransformDataset(torch.utils.data.Subset(trainds.dataset, list(range(min(4, len(trainds.dataset))))), 
                        transform=_transf),
        batch_size=2,
        collate_fn=collate_fn
    )
    
    batch = next(iter(test_loader))
    print(f"  ✓ Batch criado com sucesso!")
    print(f"    - pixel_values shape: {batch['pixel_values'].shape}")
    print(f"    - labels shape: {batch['labels'].shape}")
    print(f"    - file_ids: {batch['file_id']}")
except Exception as e:
    print(f"  ❌ ERRO ao criar batch: {e}")
    print("  Este é o mesmo erro que vai ocorrer no treinamento!")

# 4. Estimar tempo de treinamento
print("\n4️⃣  ESTIMATIVA DE TEMPO DE TREINAMENTO:")
total_samples = len(trainds)
batch_size = args.per_device_train_batch_size
num_epochs = args.num_train_epochs
total_batches = (total_samples // batch_size) * num_epochs

print(f"  Total de samples: {total_samples}")
print(f"  Batch size: {batch_size}")
print(f"  Epochs: {num_epochs}")
print(f"  Total de batches: {total_batches}")

if torch.cuda.is_available():
    time_per_batch_sec = 0.5  # Estimativa com GPU: 0.5s por batch
    estimated_hours = (total_batches * time_per_batch_sec) / 3600
    print(f"\n  ⏱️  ESTIMADO COM GPU: {estimated_hours:.1f} horas")
else:
    time_per_batch_sec = 5.0  # CPU é ~10x mais lento
    estimated_hours = (total_batches * time_per_batch_sec) / 3600
    print(f"\n  ⏱️  ESTIMADO COM CPU: {estimated_hours:.1f} horas (⚠️  MUITO LENTO!)")

print("\n" + "="*70)

In [ ]:
# ============================================================================
# CELL 15.5: OTIMIZAÇÃO ANTES DO TREINAMENTO (NOVO)
# ============================================================================
# RAZÃO: 4+ horas é inaceitável. Precisamos acelerar.
# ESTRATÉGIA: 
#   1. Forçar modelo para GPU
#   2. Usar mixed precision (fp16) se GPU disponível
#   3. Opção de reduzir dataset para teste rápido

print("="*70)
print("⚡ OTIMIZAÇÃO PRÉ-TREINAMENTO")
print("="*70)

# Forçar modelo para GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

print(f"\n✓ Modelo movido para: {device}")

# Configurar mixed precision se GPU disponível (reduz memória, aumenta velocidade)
if torch.cuda.is_available():
    print("\n✓ GPU detectada - ativando otimizações:")
    
    # Mostrar info de GPU
    print(f"  - GPU: {torch.cuda.get_device_name(0)}")
    print(f"  - VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    
    # Adicionar mixed precision aos args
    args.fp16 = True  # Usar float16 (mais rápido, menos memória)
    print(f"  - Mixed precision: ✅ ativado (fp16)")
else:
    print("\n⚠️  CPU detectada - treinamento será MUITO lento!")
    print("   Considere usar GPU (Colab, cloud, etc)")

# Opção 1: Fazer teste rápido com subset
USE_SUBSET = False  # Mude para True se quiser teste rápido
if USE_SUBSET:
    print("\n🧪 MODO TESTE (subset 10%)")
    # Usar apenas 10% dos dados para teste rápido
    subset_size = max(10, len(trainds) // 10)
    val_subset_size = max(5, len(valds) // 10)
    
    from torch.utils.data import Subset
    trainds = Subset(trainds, list(range(subset_size)))
    valds = Subset(valds, list(range(val_subset_size)))
    
    print(f"  Train: {len(trainds)} → {subset_size} samples")
    print(f"  Val: {len(valds)} → {val_subset_size} samples")
    
    # Recalcular epochs para teste (usar menos)
    args.num_train_epochs = 1
    print(f"  Epochs: 3 → 1 (teste rápido)")

print("\n" + "="*70)
print(f"⏱️  Tempo estimado AGORA: ", end="")
if torch.cuda.is_available() and not USE_SUBSET:
    print("~30-45 min (GPU otimizada)")
elif USE_SUBSET:
    print("~2-3 min (subset teste)")
else:
    print("~3-5 horas (CPU - pode interromper com Ctrl+C)")
print("="*70 + "\n")

In [ ]:
# ============================================================================
# CELL 15: TREINAR MODELO
# ============================================================================
# SEM ALTERAÇÕES: Treinamento funciona igual
# COMPORTAMENTO ESPERADO:
#   1. Salva checkpoint a cada epoch
#   2. Valida a cada epoch
#   3. Mantém melhor modelo baseado em accuracy de validação
#   4. Exibe progresso em tempo real

trainer.train()

In [ ]:
# ============================================================================
# CELL 16: FAZER PREDIÇÕES NO TEST SET (SEM LABELS)
# ============================================================================
# ALTERAÇÃO PRINCIPAL: Test set NÃO tem labels (cenário Kaggle real)
# RAZÃO: Em competições, você só faz predict sem comparar com verdade
# IMPORTANTE: Rastrear file_ids para submissão correta

# Fazer predições no test set
outputs = trainer.predict(testds)

print("Predições completadas!")
print(f"Shape das predições (logits): {outputs.predictions.shape}")
print(f"  - Dimensão 0 (samples): {outputs.predictions.shape[0]}")
print(f"  - Dimensão 1 (classes): {outputs.predictions.shape[1]}")

# Converter logits em classes preditas
predicted_classes = np.argmax(outputs.predictions, axis=1)
print(f"\nPredições (classe índice): {predicted_classes[:10]}")  # Primeiras 10

# ============================================================================
# CÓDIGO ANTIGO (COMENTADO) - Para referência
# ============================================================================
# # ORIGINAL - funcionava mas sem rastreamento de file_ids:
# outputs = trainer.predict(testds)
# print(outputs.metrics)

In [ ]:
# ============================================================================
# CELL 17: GERAR ARQUIVO DE SUBMISSÃO KAGGLE
# ============================================================================
# NOVO CELL: Não existia no código antigo (agora necessário para workflow real)
# RAZÃO: Competições Kaggle exigem arquivo no formato específico
# FORMATO: file_id, class (predito)

# Extrair file_ids do test set (mantendo ordem)
test_file_ids = []
for i in range(len(testds.dataset)):
    sample = testds.dataset[i]
    test_file_ids.append(sample['file_id'])

# Criar DataFrame de submissão
submission_df = pd.DataFrame({
    'file_id': test_file_ids,
    'class': predicted_classes
})

print("DataFrame de submissão:")
print(submission_df.head(10))
print(f"\nTotal de predições: {len(submission_df)}")
print(f"Classes preditas - distribuição:")
print(submission_df['class'].value_counts().sort_index())

# Salvar arquivo de submissão
submission_path = '/workspace/submission.csv'
submission_df.to_csv(submission_path, index=False)
print(f"\n✓ Arquivo de submissão salvo em: {submission_path}")

# Verificar arquivo criado
print("\nPrimeiras linhas do arquivo de submissão:")
print(pd.read_csv(submission_path).head(10))

In [ ]:
# ============================================================================
# CELL 18: ANÁLISE E CONFUSION MATRIX (VALIDATION SET APENAS)
# ============================================================================
# ALTERAÇÃO IMPORTANTE: Analisamos apenas VAL set (que tem labels)
# RAZÃO: Test set não tem labels verdadeiros (cenário Kaggle)
# MÉTRICA: Confusion matrix mostra performance por classe
# INTERPRETAÇÃO: Diagonal alta = bom, off-diagonal = erros entre pares de classes

# Fazer predições no validation set (que tem labels verdadeiros)
val_outputs = trainer.predict(valds)

print("Análise de Validação:")
print(f"Accuracy no validation set: {val_outputs.metrics.get('accuracy', 'N/A'):.4f}")

# Extrair predictions e labels verdadeiros
y_true = []
y_pred = []

for i in range(len(valds.dataset)):
    sample = valds.dataset[i]
    y_true.append(sample['label'])

y_true = np.array(y_true)
y_pred = np.argmax(val_outputs.predictions, axis=1)

# Criar confusion matrix
labels_list = list(itos.values())
cm = confusion_matrix(y_true, y_pred, labels=range(num_labels))

# Exibir confusion matrix
plt.figure(figsize=(12, 10))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels_list)
disp.plot(xticks_rotation=45, cmap='Blues')
plt.title(f"Confusion Matrix - Validation Set (Accuracy: {val_outputs.metrics.get('accuracy', 'N/A'):.4f})")
plt.tight_layout()
plt.show()

# Análise por classe
print("\n" + "="*70)
print("ANÁLISE POR CLASSE (Validation Set)")
print("="*70)

from sklearn.metrics import precision_recall_fscore_support

precision, recall, f1, support = precision_recall_fscore_support(
    y_true, y_pred, labels=range(num_labels), zero_division=0
)

for i in range(num_labels):
    print(f"\nClasse {i} ({labels_list[i]}):")
    print(f"  Precision: {precision[i]:.4f} (% acertos quando prediz classe i)")
    print(f"  Recall:    {recall[i]:.4f} (% classe i detectados)")
    print(f"  F1-Score:  {f1[i]:.4f}")
    print(f"  Suporte:   {support[i]} (samples)")

# ============================================================================
# CÓDIGO ANTIGO (COMENTADO) - Para referência
# ============================================================================
# # ORIGINAL - Testava no test set (que tinha labels em CIFAR-10):
# outputs = trainer.predict(testds)
# print(outputs.metrics)
# 
# itos[np.argmax(outputs.predictions[0])], itos[outputs.label_ids[0]]
# 
# y_true = outputs.label_ids
# y_pred = outputs.predictions.argmax(1)
# 
# labels = trainds.features['label'].names
# cm = confusion_matrix(y_true, y_pred)
# disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
# disp.plot(xticks_rotation=45)